# API DATA COLLECTION

This document includes the steps to connect to the Energy and Weather APIs

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import json
import requests
import os
from datetime import datetime, timedelta
os.chdir("/Users/oanottage/Desktop/BTS/BDI/Final_Project/")

In [3]:
today = datetime.today().date()
week_ago = today - timedelta(days=7)

## Data Collection

### Energy Consumption API

In [10]:
def energy_bronze():
    url = "https://api.eia.gov/v2/electricity/rto/daily-region-data/data/"
    params = {
        "api_key":"pfHsG8Kz0n6lk2qkEvOUdEfip5uJeCeeBGQqfMqa",
        "frequency": "daily",
        "data[0]": "value",
        "start": week_ago,
        "end": week_ago,
        "sort[0][column]": "period",
        "sort[0][direction]": "asc",
        "offset": "0",
        "length": "10000"
    }

    response = requests.get(url, params=params)

    # Convert response content to dictionary
    data = json.loads(response.content.decode('utf-8'))

    # Extract the data as a list of dictionaries
    data_list = data['response']['data']

    # Create pandas dataframe from the list of dictionaries
    energy_df = pd.DataFrame(data_list)
    energy_df.to_csv(f"Data_Lake/Bronze_Layer/Energy_Daily_Region_{week_ago}.csv",index=False)
    return energy_df

### Weather API

In [6]:
def weather_bronze():
  url = f"https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&locationid=FIPS:36&startdate={week_ago}&enddate={week_ago}&limit=1000"
  token = "vEExMEAzWdmnwblNYcNKBaIaasvEqEAm"

  # Using requests to send a GET request with token header
  headers = {"token": token}
  response = requests.get(url, headers=headers)

  # Load the JSON response into a dictionary
  data = json.loads(response.content.decode('utf-8'))

  # Extract the data as a list of dictionaries
  data_list = data['results']

  # Create pandas dataframe from the list of dictionaries
  weather_df = pd.DataFrame(data_list)
  weather_df.to_csv(f"Data_Lake/Bronze_Layer/Daily_Weather_{week_ago}.csv",index=False)
  return weather_df

In [7]:
weather = weather_bronze()

In [11]:
energy = energy_bronze()

In [12]:
energy

,period,respondent,respondent-name,type,type-name,timezone,timezone-description,value,value-units
0,2023-03-30,WAUW,Western Area Power Administration - Upper Grea...,D,Demand,Arizona,Arizona,2450,megawatthours
1,2023-03-30,WAUW,Western Area Power Administration - Upper Grea...,D,Demand,Central,Central,2451,megawatthours
2,2023-03-30,WAUW,Western Area Power Administration - Upper Grea...,D,Demand,Eastern,Eastern,2453,megawatthours
3,2023-03-30,WAUW,Western Area Power Administration - Upper Grea...,D,Demand,Mountain,Mountain,2454,megawatthours
4,2023-03-30,WAUW,Western Area Power Administration - Upper Grea...,D,Demand,Pacific,Pacific,2450,megawatthours
...,...,...,...,...,...,...,...,...,...
1385,2023-03-30,YAD,"Alcoa Power Generating, Inc. - Yadkin Division",TI,Total interchange,Pacific,Pacific,2011,megawatthours
1386,2023-03-30,FPL,Florida Power & Light Co.,TI,Total interchange,Arizona,Arizona,7113,megawatthours
1387,2023-03-30,FPL,Florida Power & Light Co.,TI,Total interchange,Mountain,Mountain,7421,megawatthours
1388,2023-03-30,FPL,Florida Power & Light Co.,TI,Total interchange,Pacific,Pacific,7113,megawatthours
